# COGS 108 - Data Checkpoint

# Names

- Serena Tang
- Jeffrey Luu
- Benjamin Hofflich
- Faris Ashai
- Steven Won

<a id='research_question'></a>
# Research Question

**Is there a significant difference between obesity rates in individuals across income levels in different cities in the United States for each US state?**

There is a presence of heterogenity across individual regions in the United States in terms of income and obesity rates. By identifying the obesity rates and average income for each city in the US, can we determine an index for the correlation strength between income and obesity within each city? Can we then create a ranking system to compare the indices across cities and states? Is there a geographic trend for the intensity of income/obesity correlation across the United States?

# Dataset(s)

- Dataset Name: 500 Cities: City-level Data (GIS Friendly Format), 2019 release
- Link to the dataset: https://chronicdata.cdc.gov/500-Cities-Places/500-Cities-City-level-Data-GIS-Friendly-Format-201/dxpw-cm5u
- Number of observations: 500
- Number of features: 117
- Description: The above dataset was obtained from the CDC and details an estimated percent of a city's population who has a specific health condition from 2017. Specifically, for each city, it contains the city name and FIPS code, its population count from 2010, and a state abbreviation. Additionally, it also contains a model-based estimate for the crude and age adjusted prevalence of certain health conditions and its 95% confidence interval for adults older than 18 years old; it has statistics for conditions such as those without health insurance, arthritis, binge drinking, high blood pressure and those who are taking medicine to treat it, cancer (excluding skin cancer), lung disease, heart disease, diabetes, obesity, etc. For the purpose of this project, we will mainly be using the state abbreviation, city name, and age adjusted prevalence for obesity. 


- Dataset Name: ZIP Code Data - 2017
- Link to the dataset: https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2017-zip-code-data-soi
- Number of observations: 27760
- Number of features: 153
- Description: The above dataset was obtained from the IRS and contains tax information across cities in the US. Specifically, it contains a state's abbreviation and FIPS, city zipcode, total income amount (reported in thousands of dollars), number of returns filed, salary, wages, taxable income, and other tax related information. For the purpose of this project, we will mainly be using the state, zipcode, number of returns, and total income amount. 


- Dataset Name: US Zip Codes Database
- Link to the dataset: https://simplemaps.com/data/us-zips
- Number of observations: 33120
- Number of features: 18
- Description: The above dataset was obtained from Simplemaps and contains a database of US zipcodes. For each zipcode, this dataset contains its latitude, longitude, city name, county name and FIPS and its weight, state abbreviation and name, and timezone. Additionally, if the zipcode is a Zip Code Tabulation area (ZCTA), it will contain information about its ZCTA, population, density. The database also contains information if the latitude and longitude was geolocated using the city name and if the zipcode is used by the US military. For the purpose of this project, we will mainly be using the state, city name, and zipcode information. 

The CDC dataset will be combined with the ZIP code dataset. As the CDC dataset details an estimated city's percentage of obese residences in a specific state, its information will be linked to a state's zip code and income level. This combination of datasets will allow us to understand how a city's income level correlates to the percent of residents who are obese. As the CDC dataset only contains city names and the IRS dataset contains zipcodes, the US zipcode dataset will be used to relate all the zipcodes that belong to a particular city. 

# Setup

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 100)
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
CDC_health_df = pd.read_csv('https://chronicdata.cdc.gov/api/views/dxpw-cm5u/rows.csv')
IRS_income_df = pd.read_csv('https://www.irs.gov/pub/irs-soi/17zpallnoagi.csv')
zipcodes_df = pd.read_csv('uszips.csv')
fips_to_zip_df = pd.read_csv('https://query.data.world/s/ccnj3jxdwnsm5mhahx4t52vdnusium')

# Data Cleaning

- The CDC dataset (`CDC_health_df`) will be truncated to show the city, state, and a model-based estimate for the crude and age adjusted of obesity in adults; each will be renamed for clarity. Additionally, we will drop any rows with missing information.
- The IRS dataset (`IRS_income_df`) will be truncated to show the state, zipcode, and the total income amount; each of these column names were renamed for clarity. Additionally, we will drop any rows with missing information.
- The Zipcode dataset (`zipcodes_df`) will be truncated to show the zipcode, city, and state abbreviation; each of these column names were renamed for clarity. Additionally, we will drop any rows with missing information.
- The IRS dataset and the zipcode data set were combined to map a zipcode to its city (`city_income_df`); this allows us to relate the income of different parts of a city to the entire city. Additionally, as the IRS income data is reported as the total income that zipcode makes, it was divided by the number of returns to understand the average income of a city's population.
- The city-income dataset (`city_income_df`) was then combined with the CDC dataset (`CDC_health_df`) to allow us to relate a city's income to its obesity prevalence. Finally, the average incomes (based on the zipcode) for a city was then averaged to determine a city's average income.
- The US Zip Code FIPS Lookup (`fips_to_zip_df`) will remove extra columns and be formatted to easily query a zip code using the FIPS code.

In [ ]:
# View dataset
CDC_health_df.head()

In [ ]:
# View dataset shape
CDC_health_df.shape

In [ ]:
# View dataset columns
list(CDC_health_df.columns)

In [ ]:
# Load CDC Obesity Data by State and City
CDC_health_df = CDC_health_df[['StateAbbr','PlaceName','OBESITY_CrudePrev','OBESITY_AdjPrev']]
CDC_health_df.columns = ['State','City','CrudeObesity','AdjObesity']
CDC_health_df = CDC_health_df.dropna(how = 'all')
CDC_health_df

In [ ]:
# View dataset
IRS_income_df.head()

In [ ]:
# View dataset shape
IRS_income_df.shape

In [ ]:
# View dataset columns
list(IRS_income_df.columns)

In [ ]:
#Load Income data and # of tax returns by State and Zipcode
IRS_income_df = IRS_income_df[['STATE','ZIPCODE','N1','A02650']]
IRS_income_df.columns = ['State','Zipcode','Returns','Income']
IRS_income_df = IRS_income_df.dropna(how = 'all')
IRS_income_df.head()

In [ ]:
# View dataset
zipcodes_df.head()

In [ ]:
# View dataset shape
zipcodes_df.shape

In [ ]:
# View dataset columns
zipcodes_df.columns

In [ ]:
#Load Zipcode data
zipcodes_df = zipcodes_df[['zip','city','state_id']]
zipcodes_df.columns = ['Zipcode','City','State']
zipcodes_df = zipcodes_df.dropna(how = 'all')
zipcodes_df.head()

In [ ]:
#Merge Zipcodes and Cities to get income by Zipcode
city_income_df = pd.merge(
    IRS_income_df,
    zipcodes_df, 
    on = ['Zipcode','State']
)
city_income_df.drop('Zipcode', axis = 1, inplace = True)
city_income_df.dropna(how = 'all', inplace = True)
city_income_df['Income'] = city_income_df['Income'] / city_income_df['Returns'] * 1000
city_income_df.drop('Returns', axis = 1, inplace = True)
city_income_df.head()

In [ ]:
#Merge Obesity and Income Data
income_obesity_df = pd.merge(city_income_df,CDC_health_df, on = ['State','City'])
income_obesity_df.head()

In [ ]:
#Income statistics by city (Labels not yet added)
city_income_stats = income_obesity_df.groupby(['State','City'], as_index = False).describe().Income

In [ ]:
#Average Income by city
income_obesity_df=income_obesity_df.groupby(['State','City'], as_index = False).mean()
income_obesity_df.head()

In [ ]:
#Add labels to city income statistics
city_income_stats["State"] = income_obesity_df.State
city_income_stats["City"] = income_obesity_df.City
city_income_stats = city_income_stats[["State", "City", "count", "mean", "std", "min", "25%", "50%", "75%", "max"]]
city_income_stats.head()

In [ ]:
# View dataset
fips_to_zip_df.head()

In [ ]:
# View dataset shape
fips_to_zip_df.shape

In [ ]:
# View dataset
fips_to_zip_df.columns

In [ ]:
# Remove unnecesssary columns
fips_to_zip_df = fips_to_zip_df[['ZIP', 'STCOUNTYFP', 'CITY', 'STATE']]

fips_to_zip_df = fips_to_zip_df.dropna(how = 'all')

# Given any FIPS code, we can query it and map to the corresponding ZIP code
fips_to_zip_df.set_index('STCOUNTYFP')